In [76]:
import json
with open('problematic_2_5.json', 'r+') as fp:
    problematic = json.load(fp)['problematic_2_5']
with open('combined_wordlist_2.txt', 'r') as f:
    wordlist = f.read().split('\n')[1:]
historylist = []
analytical = {}

In [65]:
print(len(wordlist))

12972


In [77]:
roundlist = []
for j in range(13):
    roundlist.append((1000*j, 1000*(j+1)))
roundlist[-1] = (12000, 12972)

In [ ]:
rounds = len(wordlist)
rounds = (2000,3000)
for rounds in roundlist[:2]:
    player = AutoWordlePlayer(wordlist, rounds, WordleSolver, 4, 5, 2, 10)
    player.play_round()
    average_guesses = player.average_guesses()
    #print(f"Average guesses per round: {average_guesses}")
    average_time = player.average_time()
    #print(f"Average time per round: {average_time}")
    #print(player.return_stats())
    for kl in player.history_history:
        historylist.append(kl)
#for kl in player.history_history:
    #print(kl)
#three_pointers = player.threes
#{3: 52, 4: 192, 5: 164, 6: 71, 7: 16, 8: 3, 9: 2}
#3: 615, 4: 2216, 5: 1517, 6: 489, 7: 141, 8: 21, 9: 1}
#3: 847, 4: 1731, 5: 1619, 6: 555, 7: 201, 8: 42, 9: 5}
#1: 1, 2: 1, 3: 2305, 4: 4610, 5: 4088, 6: 1428, 7: 427, 8: 99, 9: 12, 10: 1}
#1: 1, 2: 176, 3: 2109, 4: 3981, 5: 4448, 6: 1620, 7: 499, 8: 116, 9: 20, 10: 1, 11: 1}
#1: 1, 2: 176, 3: 2109, 4: 3731, 5: 4889, 6: 1621, 7: 362, 8: 69, 9: 12, 10: 1, 11: 1}
#1: 1, 2: 176, 3: 2118, 4: 4124, 5: 4171, 6: 1567, 7: 557, 8: 197, 9: 55, 10: 5, 11: 1}

In [69]:
length_guesses = [len(wfb) for wfb in historylist]
print(length_guesses[:2])
cnt = Counter(length_guesses)
{i: cnt[i] for i in range(15)}

[5, 3]


{0: 0,
 1: 1,
 2: 176,
 3: 2118,
 4: 2817,
 5: 5782,
 6: 1668,
 7: 339,
 8: 61,
 9: 8,
 10: 1,
 11: 1,
 12: 0,
 13: 0,
 14: 0}

In [23]:
problematic = []
for kl in player.history_history:
    if len(kl) >= 5:
        problematic.append(kl[-1][0])

In [26]:
dz = {'problematic_2_5': problematic.copy()}
import json
with open('problematic_2_5.json', 'w+') as fp:
    json.dump(dz, fp, indent=4)

In [15]:
hh = player.guess_history_output
Counter(hh).most_common(3)

[('toeas', 12972), ('riled', 962), ('lairy', 825)]

In [ ]:
n = (5, 2000)
for i in range(*n):
    print(f'{(i-n[0])/(n[1]-n[0])*100:.2f}', end='\r')
    time.sleep(0.1)
# Output: 11.90%

In [81]:
import random
from time import perf_counter_ns as pcn
class AutoWordlePlayer:
    def __init__(self, wordlist, n, agent, length_final_words, amount_of_letters, length_history, length_outlist):
        self.wordlist = wordlist
        self.current_word = random.choice(wordlist)
        self.rounds_played = 0
        self.total_guesses = 0
        self.feedback_history = []
        self.times_per_round = []
        if isinstance(n, tuple):
            self.n = n 
        elif isinstance(n, list):
            self.wordlist = n
            self.n = [0, len(self.wordlist)]
        else:
            self.n = [0, n]
        self.solving_agent = agent
        self.stats = {i:0 for i in range(20)}
        self.history_history = []
        self.threes = set()
        self.guess_history_output = []
        self.length_final_words = length_final_words
        self.amount_of_letters = amount_of_letters
        self.length_history = length_history
        self.length_outlist = length_outlist

    def enter_playmode(self):
        print(f"Entering play mode with word: {self.current_word}")

    def make_guess(self, guess):
        feedback = self._get_feedback(guess)
        self.total_guesses += 1
        self.feedback_history.append((guess, feedback))
        return (guess, feedback)

    def _get_feedback(self, guess):
        feedback = ''
        for g, w in zip(guess, self.current_word):
            if g == w:
                feedback += 'g'
            elif g in self.current_word:
                feedback += 'y'
            else:
                feedback += 'r'
        return feedback

    def play_round(self):
        for i in range(*self.n):
            self.feedback_history = []
            self.solver = self.solving_agent(True, self.length_final_words, self.amount_of_letters, self.length_history, self.length_outlist)
            self.current_word = wordlist[i]
            print(f'{(i-self.n[0])/(self.n[1]-self.n[0])*100:4.2f}|{i-self.n[0]}|{self.n[1]-self.n[0]}|{self.current_word}', end='\r')
            start = pcn()
            round_guesses = 0
            while True:
                guess = self.solver.guess(self.feedback_history)
                self.guess_history_output.append(guess)
                feedback = self.make_guess(guess)
                round_guesses += 1
                #print(f'{(i-self.n[0])/(self.n[1]-self.n[0])*100:4.2f}|{i-self.n[0]}|{self.n[1]-self.n[0]}|{self.current_word}|{guess}', end='\r')
                if feedback[1] == 'ggggg' or round_guesses >= 19:
                    self.stats[round_guesses] += 1
                    self.times_per_round.append(pcn()-start)
                    self.rounds_played += 1
                    #if round_guesses >= 6:
                    self.history_history.append(self.feedback_history.copy())
                    if round_guesses == 3:
                        self.threes.add(guess)
                    break
            #print([x[0] for x in self.feedback_history])
                
                
                
    def return_stats(self):
        deled = []
        for k,v in self.stats.items():
            if not v:
                deled.append(k)
        for x in deled:
            del self.stats[x]
        return self.stats
    def average_guesses(self):
        if self.rounds_played == 0:
            return 0
        return self.total_guesses / self.rounds_played
    def average_time(self):
        if self.rounds_played == 0:
            return 0
        return sum(self.times_per_round) / self.rounds_played / 1_000_000_000


In [75]:
from collections import Counter
import itertools
class WordleSolver:
    with open('combined_wordlist_2.txt', 'r') as f:
        read_wordlist = f.read().split('\n')[1:]
        all_words = {c: {i: set() for i in range(5)} for c in 'abcdefghijklmnopqrstuvwxyz'}
        for c in 'abcdefghijklmnopqrstuvwxyz':
            all_words[c]['all'] = set()
        zzz = set()
        for word in read_wordlist:
            for idx,c in enumerate(word):
                all_words[c][idx].add(word)
                all_words[c]['all'].add(word)
        possible_words = set()
        for c in 'abcdefghijklmnopqrstuvwxyz':
            possible_words |= all_words[c]['all']

    def __init__(self, hard_mode=True, length_final_words=4, amount_of_letters=5, length_history=2, length_outlist=10):
        self.letters = {c: {i: set() for i in range(5)} for c in ('g', 'y', 'r')}
        #self.letters['g'] = {i: set() for i in range(5)}
        #self.letters['y'] = {i: set() for i in range(5)}
        #self.letters['r'] = {i: set() for i in range(5)}
        self.possible_words = set()
        self.green_words = set()
        self.yellow_words = WordleSolver.possible_words.copy()
        self.red_words = set()
        self.hard_mode = hard_mode
        self.length_final_words = length_final_words
        self.amount_of_letters = amount_of_letters
        self.length_history = length_history
        self.length_outlist = length_outlist
        self.y_letter_removal_guess = 0

    def guess(self, history):
        if not history:
            return 'toeas'
        else:
            #for unpack in history:
                #for w,f,i in zip(*unpack, range(5)):
                    #self.letters[f][i].add(w)
            for ch, fb, idx in zip(*history[-1], range(5)):
                self.letters[fb][idx].add(ch)
                if fb == 'g':
                    self.update_green_words(ch, idx)
                    pass
                elif fb == 'y':
                    self.update_yellow_words(ch, idx)
                    pass
                else:
                    self.update_red_words(ch, idx)
                    pass
        
        if self.green_words:
            if self.hard_mode:
                final_words = (self.yellow_words & self.green_words) - self.red_words
            else:
                final_words = (self.yellow_words - self.green_words) - self.red_words
        else:
            final_words = self.yellow_words - self.red_words
        
        if len(final_words) > self.length_final_words and len(set(''.join(final_words))) > self.amount_of_letters and len(history) > self.length_history:
            output_guess = self.get_fitting_letters(final_words)[0]
        else:
            out = self.analysis_hard_mode(final_words)
            outlist = [c[0] for c in out[:self.length_outlist]]
            output_guess = sorted(outlist, reverse=True, key=lambda x: sum(self.ranked_word[x]))[0]
        return output_guess

    
    def update_green_words(self, letter, position):
        if self.green_words:
            self.green_words &= WordleSolver.all_words[letter][position]
        else:
            self.green_words = WordleSolver.all_words[letter][position].copy()
    
    def update_yellow_words(self, letter, position):
        if self.green_words:
            self.yellow_words &= self.green_words
        self.yellow_words &= (WordleSolver.all_words[letter]['all'] - WordleSolver.all_words[letter][position])

    def update_red_words(self, letter, position):
        self.red_words |= WordleSolver.all_words[letter]['all']
    

    def analysis_hard_mode(self, words):
        breakdown = Counter(''.join(words))
        dictio = {word: sum([breakdown[c] for c in set(word)]) for word in words}
        sorted_words = sorted([(k,v) for k,v in dictio.items()], reverse=True, key=lambda x: x[1])
        self.analysis(words)
        return sorted_words

    def analysis(self, words):
        ranking = {i: '' for i in range(5)}
        for word in words:
            for idx, c in enumerate(word):
                ranking[idx] += c
        for k,v in ranking.items():
            ranking[k] = Counter(v)
        self.ranked_word = {word: [ranking[idx][c] for idx, c in enumerate(word)] for word in words}

    def get_fitting_letters(self, words):
        joined_words = ''.join(words)
        self.fitting = set(joined_words)
        rank_it = Counter(joined_words)
        
        potential_words = set()
        for l in self.fitting:
            potential_words |= WordleSolver.all_words[l]['all']

        for c in set.union(*self.letters['g'].values()):
            rank_it[c] = 0
        for c in set.union(*self.letters['y'].values()):
            rank_it[c] = int(rank_it[c]/(100-self.y_letter_removal_guess))
        
        #for x in self.letters['y'].values():
            #self.fitting -= x
        
        dictio = {w: sum([rank_it[l] for l in set(w) if l in self.fitting]) for w in potential_words}
        self.guesses_ = list(sorted(potential_words, reverse=True, key=lambda x: dictio[x]))
        #print(self.guesses_[0], dictio[self.guesses_[0]])
        self._guesses_ = list(itertools.takewhile(lambda x: dictio[x]>=dictio[self.guesses_[0]], self.guesses_))
        #print(len(self._guesses_), len(self.guesses_))
        return self.guesses_
        #return self.analysis_hard_mode(_guesses_)
        
        

In [ ]:
length_final_words, amount_of_letters, length_history